In [ ]:
import numpy as np

In [ ]:
class params: 
    def __init__(self, ):
        self

class test_sampler:
    """test sampler"""
    samples = []
    def __init__(self, params, T):
        self.params = params
        self.T = T 
    def sample(self, sample_num:int):
        samples = np.zeros((self.T, sample_num))

    def policy(self, rt, rt1, et1):
        """sampling policy"""
        return et1 + np.random.randn()

    def policy_density(self, et, rt, rt1, et1): 
        """pdf of policy"""
        return np.exp(-(et-et1)**2/2)      